In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import time
import re
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
Base = declarative_base()
import pymysql
pymysql.install_as_MySQLdb()
import numpy as np

# Set path 
import sys
sys.path.append('../')

# Config variables
from config import remote_db_endpoint, remote_db_port
from config import remote_gwsis_dbname, remote_gwsis_dbuser, remote_gwsis_dbpwd
from config import local_gwsis_dbname, local_gwsis_dbuser, local_gwsis_dbpwd 

Create local and remote engines and pass in MySQL connection

In [2]:
# AWS Database Connection
engine = create_engine(f"mysql://{remote_gwsis_dbuser}:{remote_gwsis_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_gwsis_dbname}")

In [3]:
# Create a remote database engine connection
conn = engine.connect()

In [4]:
# Use Splinter to initiate browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
#['depo-provera','mirena','nuvaring','ortho-tri-cyclen','norethindrone','nexplanon','xulane','paragard']
types_list = ['medroxyprogesterone/depo-provera-','levonorgestrel/mirena-','ethinyl-estradiol-etonogestrel/nuvaring-',
              'ethinyl-estradiol-norgestimate/','ethinyl-estradiol-norethindrone/','etonogestrel/nexplanon-',
              'ethinyl-estradiol-norelgestromin/xulane-','copper-topical/']

Scrape all the reiviews from [Drugs.com](https://www.drugs.com/) for each of the birth control types. This may take several minutes!

In [6]:
bc_type = []
review_text = []
stars = []
publish_date = []

for t in types_list:
    # URL of page to be scraped (last scraped 3/30/2019)
    url = f'https://www.drugs.com/comments/{t}for-contraception.html'
    browser.visit(url)
    
    # Limit it to 13 pages or reviews for each or else it will take forever
    for x in range(1, 13):

        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        # Retrieve the parent divs for all reviews
        results = soup.find_all('div', class_='user-comment')

        # Loop through results to retrieve review details
        for result in results:
            body = result.span.text
            bc = t 
            try:
                #edited tags on 3/30 to reflect website updates
                date = result.find('p', class_="text-size-smaller user-name user-type user-type-2_non_member").find_all('b')[1].text
                publish_date.append(date)  
            except:
                publish_date.append('NA')  
    
            bc_type.append(bc)
            review_text.append(body)  
            
            try:
                rate = result.tr.text.strip()
                rating = int(float(rate))
                stars.append(rating)
            except:                
                stars.append('NA')
        try:
            browser.click_link_by_partial_text('Next')

            time.sleep(1)

        except:
            print("Scraping Complete")

In [7]:
## Number of reviews collected
len(publish_date)

2400

In [8]:
bc_df_2 = pd.DataFrame({
    'Birth_Control': bc_type,
    'Star_Rating': stars,
    'Review': review_text,
    'Publish_Date': publish_date
    })

In [9]:
bc_df_2.head()

,Birth_Control,Star_Rating,Review,Publish_Date
0,medroxyprogesterone/depo-provera-,1,"""I would rate this drug 0 out of 10 if possibl...","April 1, 2019"
1,medroxyprogesterone/depo-provera-,8,"""Overall 8/10. I give it 10/10 for effectivene...","March 31, 2019"
2,medroxyprogesterone/depo-provera-,1,"""The Deop-provera Brith control shot is the wo...","March 14, 2019"
3,medroxyprogesterone/depo-provera-,5,"""The Depo shot has been very effective. I have...","March 10, 2019"
4,medroxyprogesterone/depo-provera-,1,"""Depo provera was awful. I gained over 35 poun...","March 7, 2019"


Use VADER sentiment analysis to analyze the review text for each review. How do women feel about their birth control?

In [10]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [11]:
# Variables for holding sentiments
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

for review in bc_df_2['Review']:

    # Run Vader Analysis on each review
    results = analyzer.polarity_scores(review)
    compound = results["compound"]
    pos = results["pos"]
    neu = results["neu"]
    neg = results["neg"]

    # Add each value to the appropriate list
    compound_list.append(compound)
    positive_list.append(pos)
    negative_list.append(neg)
    neutral_list.append(neu)

Store it all in a DataFrame, add original data source and birth control type.

In [12]:
# Store it all in a DataFrame
bc_df_2 = pd.DataFrame({
    'Birth_Control': bc_type,
    'Star_Rating': stars,
    'Review': review_text,
    'Use': 'n/a',
    'Publish_Date': publish_date,
    'Compound': compound_list,
    'Positive': positive_list,
    'Negative': negative_list,
    'Neutral': neutral_list, 
    'Source' : 'Drugs.com'
    })

In [14]:
# Add type based on the birth control product
for index, row in bc_df_2.iterrows():
    if row['Birth_Control'] == 'medroxyprogesterone/depo-provera-':
        bc_df_2.loc[index,"Method"] = 'Shot'
    elif row['Birth_Control'] == 'levonorgestrel/mirena-':
        bc_df_2.loc[index,"Method"] = 'Hormonal IUD'
    elif row['Birth_Control'] == 'etonogestrel/nexplanon-':
        bc_df_2.loc[index,"Method"] = 'Implant'
    elif row['Birth_Control'] == 'ethinyl-estradiol-norethindrone/':
        bc_df_2.loc[index,"Method"] = 'Progestin Pill'
    elif row['Birth_Control'] == 'ethinyl-estradiol-etonogestrel/nuvaring-':
        bc_df_2.loc[index,"Method"] = 'Ring'
    elif row['Birth_Control'] == 'ethinyl-estradiol-norelgestromin/xulane-':
        bc_df_2.loc[index,"Method"] = 'Patch'
    elif row['Birth_Control'] == 'ethinyl-estradiol-norgestimate/':
        bc_df_2.loc[index,"Method"] = 'Combination Pill'
    elif row['Birth_Control'] == 'copper-topical/':
        bc_df_2.loc[index,"Method"] = 'Non-hormonal IUD'    

Clean up the product names

In [15]:
bc_df_2['Birth_Control'] = bc_df_2['Birth_Control'].replace({'medroxyprogesterone/depo-provera-': 'Depo-Provera',\
                                                             'levonorgestrel/mirena-':'Mirena',\
                                                             'etonogestrel/nexplanon-': 'Implanon',\
                                                             'ethinyl-estradiol-norethindrone/': 'Norethindrone',\
                                                             'ethinyl-estradiol-etonogestrel/nuvaring-': 'NuvaRing',\
                                                             'ethinyl-estradiol-norelgestromin/xulane-':'Xulane',\
                                                             'ethinyl-estradiol-norgestimate/':'Ortho Tri-Cyclen',\
                                                             'copper-topical/': 'ParaGard'
                                                            })

In [16]:
bc_df_2.head(100)

,Birth_Control,Star_Rating,Review,Use,Publish_Date,Compound,Positive,Negative,Neutral,Source,Method
0,Depo-Provera,1,"""I would rate this drug 0 out of 10 if possibl...",n/a,"April 1, 2019",-0.8210,0.065,0.156,0.780,Drugs.com,Shot
1,Depo-Provera,8,"""Overall 8/10. I give it 10/10 for effectivene...",n/a,"March 31, 2019",0.9639,0.150,0.009,0.840,Drugs.com,Shot
2,Depo-Provera,1,"""The Deop-provera Brith control shot is the wo...",n/a,"March 14, 2019",0.8650,0.125,0.083,0.792,Drugs.com,Shot
3,Depo-Provera,5,"""The Depo shot has been very effective. I have...",n/a,"March 10, 2019",-0.8519,0.072,0.180,0.748,Drugs.com,Shot
4,Depo-Provera,1,"""Depo provera was awful. I gained over 35 poun...",n/a,"March 7, 2019",0.1899,0.136,0.099,0.765,Drugs.com,Shot
5,Depo-Provera,1,"""I did the depo shots for about a year and a h...",n/a,"February 11, 2019",-0.0176,0.111,0.103,0.786,Drugs.com,Shot
6,Depo-Provera,6,"""It did what it was supposed to which is stop ...",n/a,"February 5, 2019",0.9736,0.201,0.094,0.705,Drugs.com,Shot
7,Depo-Provera,5,"""I got the shot once and it was TERRIBLE. I ex...",n/a,"February 3, 2019",-0.9267,0.082,0.320,0.598,Drugs.com,Shot
8,Depo-Provera,10,"""My experience with this shot has been nothing...",n/a,"January 29, 2019",0.6826,0.153,0.107,0.741,Drugs.com,Shot
9,Depo-Provera,5,"""I got on the Depo shot January 2016. I gained...",n/a,"January 14, 2019",-0.8221,0.121,0.151,0.728,Drugs.com,Shot


In [17]:
method_count = pd.DataFrame(bc_df_2['Method'].value_counts())
method_count

,Method
Non-hormonal IUD,300
Patch,300
Implant,300
Ring,300
Shot,300
Progestin Pill,300
Combination Pill,300
Hormonal IUD,300


In [19]:
compound_score = []

for score in bc_df_2['Compound']:
    if score <= 0-.6: 
        compound_score.append('Very Negative')
    elif score <= 0-.2: 
        compound_score.append('Negative')
    elif score <= .2: 
        compound_score.append('Neutral')
    elif score <= .6: 
        compound_score.append('Positive') 
    else:
        compound_score.append('Very Positive')

In [20]:
bc_df_2['Vader_Scale'] = compound_score

In [21]:
vader_count = pd.DataFrame(bc_df_2['Vader_Scale'].value_counts())
vader_count

,Vader_Scale
Very Negative,1112
Very Positive,581
Negative,274
Positive,253
Neutral,180


In [22]:
bc_df_2.to_csv('birth_control_site2.csv')

Add our newly collected data to our existing birth control database on AWS. 

In [ ]:
bc_df_2.to_sql(name='birth_control_all', if_exists='append', con=conn, index=False)

In [24]:
# Check remote database to make sure it migrated correctly
remote_bc_data = pd.read_sql("SELECT * FROM birth_control_all", conn)
print(len(remote_bc_data))
remote_bc_data.head()

5297


,Birth_Control,Star_Rating,Review,Use,Publish_Date,Compound,Positive,Negative,Neutral,Source,Method,Vader_Scale
0,Depo-Provera,1,"After 6 years of taking this injection, my wif...",Birth Control,4/1/2019,-0.5574,0.031,0.075,0.895,Everyday Health,Shot,Negative
1,Depo-Provera,5,Have endrometrisos was bleeding from vagina an...,Endometriosis,3/10/2019,-0.6908,0.051,0.133,0.816,Everyday Health,Shot,Very Negative
2,Depo-Provera,1,I had a miscarriage due to the depo shot. Plea...,Birth Control,3/8/2019,0.0516,0.144,0.109,0.747,Everyday Health,Shot,Neutral
3,Depo-Provera,3,I have been on this for 4/5 years i would reco...,Birth Control,3/6/2019,0.7251,0.212,0.000,0.788,Everyday Health,Shot,Very Positive
4,Depo-Provera,1,"Ive been on it for 2 months now , I get Horrib...",Birth Control,3/5/2019,-0.8772,0.000,0.181,0.819,Everyday Health,Shot,Very Negative


In [25]:
# Check that long string was not truncated during transfer
print(remote_bc_data['Review'][5000])

"The insertion hurt like crazy, but maybe that's all IUD's. My period was much heavier and longer for at least 2 days, on top of my normal 7 day period. I had non-stop spotting in between cycles too. Then, after 1 year and 2 months of occasional intercourse,  with Paragard I got pregnant! 

Would not recommend this as a birth control option, unless you really like heavier, longer, and more painful periods and unplanned pregnancies. Use ANYTHING else, trust me. I'm now using a hormonal IUD and pills and haven't gotten pregnant since."
